In [1]:
#!pip install nuscenes-devkit &> /dev/null 
#!pip install pyquaternion &> /dev/null

In [2]:
# Required Imports TODO(PARTH ): CLEAN IT UP 
import os
import json
import numpy as np
from pyquaternion import Quaternion
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.geometry_utils import transform_matrix
from nuscenes.can_bus.can_bus_api import NuScenesCanBus
import torch 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
print(device)

In [4]:
data_path="/home/parth/Desktop/NortheasternCourses/MobileRobotics/FinalProject/Multicam_EgoMotion/nuscenes_mini"

In [5]:
class NuScenesDataLoader:
    """_summary_ NUSCENES DATA LOADER CLASS TO INTRACT WITH NU SCENES 
    """
    def __init__(self, dataset_path, version="v1.0-mini"):
        """_summary_ INIT METHOD IS REPONSIBLE TO initialize class variables for data loader

        Args:
            dataset_path (_type_): _description_ relative path to the dataset , if any path issue is there preferably give absolute path 
            version (str, optional): _description_. version number Defaults to "v1.0-mini".
        """
        self.dataset_path = dataset_path
        self.nusc = NuScenes(version=version, dataroot=dataset_path, verbose=True)
        self.nusc_can = NuScenesCanBus(dataroot=dataset_path)
        
    def load_data(self):
        """_summary_ This method is reponsible to create a list of data sample with the required data format from the whole nu scenes data set 
        

        Returns:
            _type_: _description_ list of dictionary of above format 
        """
        
   
        
        data_scene={}
        
        for scene in self.nusc.scene:
            
            sample_token=scene["first_sample_token"]   
            scene_name =scene["name"] 
            data_dict = {}
            intrinsics=[]
            extrinsics=[]
            sample=self.nusc.get('sample', sample_token)
            sample_data_tokens = {key: value  for key, value in sample["data"].items() if 'CAM' in key}
            for cam, token in sample_data_tokens.items():
                sd_record = self.nusc.get('sample_data', token)
                cs_record = self.nusc.get('calibrated_sensor', sd_record['calibrated_sensor_token'])
                intrinsics.append(cs_record['camera_intrinsic'])
                extrinsics.append(transform_matrix(translation=cs_record['translation'], rotation=Quaternion(cs_record['rotation']), inverse=False))
                data_dict[cam]=[]
            
            data_dict["scene"]=scene_name
            data_dict["intrinsics"]=intrinsics
            data_dict["extrinsics"]=extrinsics
            
            data_dict["pos"]=[]
            data_dict["orientation"]=[]
            data_dict["rotation_rate"]=[]
            data_dict["vel"]=[]
            data_dict["timestamp"]=[]
            data_dict["num_of_samples"]=0
                
            sample=self.nusc.get('sample', sample_token)
            sensor_suite=sample["data"]
            
            sample_data_tokens = {key: value  for key, value in sensor_suite.items() if 'CAM' in key}
            
            
            record_time_stamp=True
            for sensor,init_token in sample_data_tokens.items():
                sensor_token=init_token
                
                while sensor_token != "" :
                    sd_record = self.nusc.get('sample_data', sensor_token)
                    data_dict[sensor].append(self.nusc.get('sample_data', sensor_token)['filename'])
                    
                    if record_time_stamp:
                        data_dict["timestamp"].append(sd_record["timestamp"])
                        
                    sensor_token=sd_record["next"]
                else:
                    record_time_stamp=False # record timestep only once
            
            else:
            
                sample_token=sample["next"]
        

            for timestamp in data_dict['timestamp']:
                closest_entry = min(self.nusc_can.get_messages(scene_name, 'pose'), key=lambda x: abs(x['utime'] -timestamp))
                #print(closest_entry)
                data_dict["rotation_rate"].append(closest_entry["rotation_rate"]) 
                data_dict["vel"].append(closest_entry["vel"])
                data_dict["pos"].append(closest_entry["pos"])
                data_dict["orientation"].append(closest_entry["orientation"])
            
            data_dict["num_of_samples"]=min(len(data_dict['timestamp']),200)
            data_list=[]
            print(data_dict["num_of_samples"])
            for i in range(data_dict["num_of_samples"]):
                #print(i)
                data_list.append({
                    "id":i,
                    "scene_name":data_dict["scene"],
                    "timestamp":data_dict["timestamp"][i],
                    "vel":data_dict["vel"][i],
                    "rotation_rate":data_dict["rotation_rate"][i],
                    "pos":data_dict["pos"][i],
                    "orientation":data_dict["orientation"][i],
                    "image_list":[data_dict[key][i] for key, value in sample_data_tokens.items() if 'CAM' in key],
                    "extrinsics": data_dict["extrinsics"],
                    "intrinsics":data_dict["intrinsics"],
                    
                                  })
            
            data_scene[scene_name]=data_list
            
        return data_scene



In [ ]:


data_loader = NuScenesDataLoader(data_path)
data = data_loader.load_data()
print(len(data))
# Print a sample of the loaded data
#print(json.dumps(data[0], indent=2))

In [73]:
#dir(data_loader.nusc)

In [ ]:
#print("Data Format",data[0])
print(data_loader.nusc.scene[0])

In [8]:
import cv2

In [267]:

def dense_optical_flow(img1, img2,img_vis=False):
    rgb_flow = None
    flow = None
    flow = cv2.calcOpticalFlowFarneback(img1, img2, None, 0.5, 5, 15, 8, 7, 1.5, 0)
    #flow = cv2.optflow.createOptFlow_PCAFlow()
    #flow = cv2.optflow.createOptFlow_DeepFlow()

    # Compute optical flow
    #flow = flow.calc(img1, img2, None)
    #return flow
    
    if img_vis:
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        hsv = np.zeros_like(cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR))
        hsv[..., 0] = angle * 180 / np.pi / 2
        hsv[..., 1] = 255
        hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        rgb_flow = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return flow,rgb_flow

    


In [10]:
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline

In [11]:
#!pip install opencv-contrib-python

In [12]:
# import cv2
# cv2.__version__

In [ ]:
scene1=list(data.keys())[0]
img_t0=cv2.imread(data_path+"/"+ data[scene1][1]["image_list"][0],0)
img_t1=cv2.imread(data_path+"/"+data[scene1][2]["image_list"][0],0)
plt.imshow(img_t1)


In [ ]:
plt.imshow(img_t0)

In [ ]:
flow,rgb_flow= dense_optical_flow(img_t0,img_t1,img_vis=True)
print(flow.shape,rgb_flow.shape)
plt.imshow(rgb_flow)


In [ ]:
print("Data Frame Time Diff",(data[scene1][7]['timestamp']-data[scene1][6]["timestamp"])/1e6)


In [17]:
def resize_image(image, width, height):
    return cv2.resize(image, (width, height))

# Function to create a row of images
def create_row(images, width, height):
    resized_images = [resize_image(img, width, height) for img in images]
    return np.hstack(resized_images)

In [268]:
exec=True

if exec:

    for i in range(len(data[scene1])-2):
        optical_flow_images=[]
        imgs_t0=[]
        imgs_t1=[]
        for j in range(len(data[scene1][i]["image_list"])):
            img_t0=cv2.imread(data_path+"/"+ data[scene1][i]["image_list"][j],0)
            img_t1=cv2.imread(data_path+"/"+data[scene1][i+1]["image_list"][j],0)
            flow,rgb_flow= dense_optical_flow(img_t0,img_t1,img_vis=True)
            optical_flow_images.append(rgb_flow)
            imgs_t0.append(np.dstack([img_t0,img_t0,img_t0]))
            imgs_t1.append(np.dstack([img_t1,img_t1,img_t1]))
        num_images = 6
        image_width = 200  # Adjust as needed
        image_height = 200  # Adjust as needed

        # Create rows
        row_t0 = create_row(imgs_t0, image_width, image_height)
        row_t1 = create_row(imgs_t1, image_width, image_height)
        row_flow = create_row(optical_flow_images, image_width, image_height)

        # Stack rows vertically to form a grid
        final_grid = np.vstack([row_t0, row_t1, row_flow])
        
        cv2.imwrite("./results/image_grid{}.jpg".format(i), final_grid) # update this file path to change directory of writing results 
    
    
        
        
        

In [19]:

# Sparse Optical Flow using Lucas-Kanade method
def sparse_optical_flow(img1, img2,img_vis=False):
    feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
    lk_params = dict(winSize=(15, 15), maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    p0 = cv2.goodFeaturesToTrack(img1, mask=None, **feature_params)
    p1, st, err = cv2.calcOpticalFlowPyrLK(img1, img2, p0, None, **lk_params)
    #print(p1.shape,st.shape,err.shape)
    # Draw the tracks
    mask = np.zeros_like(cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR))
    flow = np.zeros_like(np.dstack([img1,img1]))
    
    for i, (new, old) in enumerate(zip(p1[st == 1], p0[st == 1])):
        a, b = new.ravel()
        c, d = old.ravel()
        flow[int(d),int(c),0]= a-c
        flow[int(d),int(c),1]= b-d
            
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color=(255, 255, 0), thickness=3)
        mask = cv2.circle(mask, (int(c), int(d)), 10, color=(255, 0, 0), thickness=-1)
    return flow,mask
    

In [ ]:
flow,mask= sparse_optical_flow(img_t0,img_t1,img_vis=True)
bin_mask=np.where(mask[:,:,0]>0,False,True)
print(flow.shape,mask.shape)
plt.imshow(bin_mask*img_t0)

In [21]:
# Write code for optical of each image

# Function 
"""_summary_
    input: 
    images_t0 : K_cameras, H,W
    mages_t1 : K_cameras, H,W
    output K_images,N_flow_vector,2  where N_flow_vectors : H*W  if dense optical flow of the image else N_flow_vectors< W*H else  sparese optical flow 
"""
def compute_optical_flow(images_t0,images_t1, dense=False,vis=True,data_path=""):
    """_summary_

    Args:
        images_t0 (_type_): _description_
        images_t1 (_type_): _description_
        dense (bool, optional): _description_. Defaults to False.
    """
    flows=None
    masks=None
    for img1_name, img2_name in zip(images_t0,images_t1):
        img_t0 =cv2.imread(os.path.join(data_path,img1_name),0)
        img_t1 =cv2.imread(os.path.join(data_path,img2_name),0)
        
        if dense:
            flow,mask=dense_optical_flow(img_t0,img_t1,vis)
            #print("In Dense",flow.shape)
        else:
            flow,mask=sparse_optical_flow(img_t0,img_t1,vis)
        
        if flows is not None or masks is not None:
            flows=np.concatenate((flows,flow[np.newaxis,...]),axis=0)
            if vis:
                masks=np.concatenate((masks,mask[np.newaxis,...]),axis=0)
        else:
            
            flows=flow[np.newaxis,...]
            
            if vis:
                
                masks=mask[np.newaxis,...]
           
    return flows,masks

In [ ]:
flows,rgb_masks=compute_optical_flow(data[scene1][0]["image_list"],data[scene1][1]["image_list"],dense=True,data_path="/home/parth/Desktop/NortheasternCourses/MobileRobotics/FinalProject/Multicam_EgoMotion/nuscenes_mini")
print(flows.shape,rgb_masks.shape)

In [23]:
#Prepocessing or utility 

#Dunction convert poitns from pixel plane to normalized camera frame and thresh it at the same time 

import numpy as np
def clean_optical_flow_to_normalized_plane_multiple_cameras(flow, intrinsic_matrices,mag_thresh=0.001):
    """
    Converts optical flow to the normalized image plane for multiple cameras.

    Parameters:
        flow (numpy.ndarray): Optical flow array of shape (K, H, W, 2), where:
                              - K is the number of cameras.
                              - H, W are the image dimensions.
                              - 2 represents horizontal and vertical flow components.
        intrinsic_matrices (numpy.ndarray): Intrinsic matrices of shape (K, 3, 3), where:
                                            - K is the number of cameras.
                                            - Each 3x3 matrix corresponds to the intrinsic parameters.

    Returns:
        numpy.ndarray: Normalized optical flow array of shape (K, H, W, 2).
    """
    # Validate input dimensions
    if len(flow.shape) != 4 or len(intrinsic_matrices.shape) != 3:
        raise ValueError("Flow must have shape (K, H, W, 2) and intrinsics (K, 3, 3).")
    if flow.shape[0] != intrinsic_matrices.shape[0]:
        raise ValueError("Number of cameras in flow and intrinsics must match.")
    
    # Extract dimensions
    K, H, W, _ = flow.shape

    # Prepare outputs
    normalized_flow = np.zeros_like(flow)
    normalized_coords = np.zeros_like(flow)
    mask =np.zeros((K, H, W), dtype=bool)

    # Loop through each camera to process flow and intrinsics
    for i in range(K):
        # Get intrinsic matrix for the current camera
        intrinsic_matrix = intrinsic_matrices[i]
        K_inv = np.linalg.inv(intrinsic_matrix)
        
        # Create grid of pixel coordinates for the current image
        x_coords, y_coords = np.meshgrid(np.arange(W), np.arange(H))
        ones = np.ones_like(x_coords)

        # Homogeneous pixel coordinates
        pixel_coords = np.stack((x_coords, y_coords, ones), axis=-1)  # Shape: (H, W, 3)
        pixel_coords_flat = pixel_coords.reshape(-1, 3)  # Shape: (H*W, 3)

        # Transform pixel coordinates to normalized coordinates
        normalized_coords_flat = (K_inv @ pixel_coords_flat.T).T  # Shape: (H*W, 3)
        normalized_coords[i,...] = normalized_coords_flat[:, :2].reshape(H, W, 2)  # Shape: (H, W, 2)
        

        # Normalize optical flow for this camera
        #dx_normalized = flow[i, ..., 0] / intrinsic_matrix[0, 0]  # Horizontal flow / fx
        #dy_normalized = flow[i, ..., 1] / intrinsic_matrix[1, 1]  # Vertical flow / fy
        dx_normalized = (flow[i, ..., 0] ) / intrinsic_matrix[0, 0]  # Horizontal flow
        dy_normalized = (flow[i, ..., 1]) / intrinsic_matrix[1, 1] 
        
        magnitude = np.sqrt(flow[i,..., 0]**2 + flow[i,..., 1]**2)
        #print(magnitude.shape)
        non_zero_magnitude=magnitude[magnitude > mag_thresh]
        
        lower_threshold = np.max([np.mean(non_zero_magnitude) - 0.5 * np.std(non_zero_magnitude),mag_thresh])
        upper_threshold = np.mean(non_zero_magnitude) + 3 * np.std(non_zero_magnitude)
        #print(upper_threshold,lower_threshold)
        clean_flow_mask = (magnitude > lower_threshold) &(magnitude <upper_threshold)
        mask[i]=clean_flow_mask
        
        normalized_flow[i, ..., 0] = dx_normalized
        normalized_flow[i, ..., 1] = dy_normalized

    return normalized_coords,normalized_flow ,mask
 

In [ ]:
normalized_coords,normalized_flow,clean_flow_mask=clean_optical_flow_to_normalized_plane_multiple_cameras(flows,intrinsic_matrices=np.array(data[scene1][0]["intrinsics"]))

print(normalized_coords.shape,normalized_flow.shape,clean_flow_mask.shape)

In [ ]:
img_mask=np.ones_like(clean_flow_mask)*clean_flow_mask
for img in clean_flow_mask:
    print(np.sum(img),img.shape[0]*img.shape[1])


In [ ]:

plt.imshow(rgb_masks[3],cmap="gray")  # Show the image
plt.imshow(clean_flow_mask[3], cmap='jet',alpha=0.1)  # Overlay the mask


In [27]:
def non_zero_flow(normalized_flow):
    mask= np.where((normalized_flow[..., 0]**2 + normalized_flow[..., 1]**2),True,False)
    return mask

    

In [ ]:
mask=non_zero_flow(normalized_flow)
masked_normalized_coords=normalized_coords[mask]
masked_normalized_flow=normalized_flow[mask]
print(len(mask),mask[0].shape, np.sum(mask[0]),900*1600)
print(masked_normalized_coords.shape,masked_normalized_flow.shape)

In [28]:
#!pip install theseus-ai

In [30]:
import torch
import theseus as th
#IGNORE THE THESEUS PART 

In [ ]:
print(th.__version__)

In [300]:
# TODO ( PARTH ) : WRITE GETTER AND SETTERS 
class MultiCamEgoMotion(object):
    def __init__(self):
    
        #super().__init__(self)
        self.flow = None  # [K, H, W, 2]
        self.normalized_coords = None # [K, H, W, 2]
        self.R_k_list = None  # [K, 3, 3]
        self.b_k_list = None  # [K, 3]
        self.degenrate_case=False
        self.M = torch.zeros((3, 3),device=device)
        self.mi = None # [K, H, W, 3]
        self.hk= None
        self.c = torch.zeros((3),device=device)
        #self.w = torch.zeros((3))a
        self.t = torch.zeros((3),device=device)
        self.w_=torch.zeros((3),device=device)

    def compute_data(self,normalized_coords, flows, Extriniscs,w,masks):
         # Extract optimization variable w
        
        if self.mi is None:
            self.mi = torch.zeros(torch.concat([normalized_coords,normalized_coords[...,:1]],axis=-1).shape,dtype=torch.float32,device=device) # [K, H, W, 3]
            self.hk =torch.zeros(torch.as_tensor(Extriniscs[:,3,0:3]).shape,dtype=torch.float32,device=device) #K,3 dims of bk and hk match but serve diff purpose 
        self.flow = flows  # [K, H, W, 2]
        self.normalized_coords = normalized_coords # [K, H, W, 2]
        self.R_k_list = torch.as_tensor(Extriniscs[:,0:3,0:3].clone().detach(),dtype=torch.float32,device=device)  # [K, 3, 3]
        self.b_k_list = torch.as_tensor(Extriniscs[:,:3,3].clone().detach(),dtype=torch.float32,device=device)  # [K, 3]
        self.w_=w # [3,1]
        
        
        # Initialize M and c
        M = torch.zeros((3, 3), dtype=torch.float32,device=device)
        c = torch.zeros((3,1), dtype=torch.float32,device=device)

        # Loop through all cameras
        for k in range(self.flow.shape[0]):  # Iterate over K cameras
            
            normalized_coords_k=self.normalized_coords[k] # [H, W, 2]
            flow_k=self.flow[k] # [H, W, 2]
            
            R_k = torch.as_tensor(self.R_k_list[k],dtype=torch.float32,device=device)  # [3, 3]
            b_k = self.b_k_list[k]  # [3]

            # Reconstruct 3D points assuming unit depth
            P_k = torch.cat([normalized_coords_k,torch.ones_like(normalized_coords_k[...,:1],device=device)],dim=-1)  # [H, W, 3]
            P_k_dot = torch.cat([flow_k, torch.zeros_like(flow_k[..., :1])], dim=-1)  # [H, W, 3]

            # Compute omega_k and h_k
            omega_k = torch.matmul(R_k.T, w.T).squeeze(-1)  # [3]
            
            h_k = torch.cross(w, b_k[None])  # [3]
            inst_rotation_action= torch.cross(omega_k[None,None].expand_as(P_k),P_k,dim=-1) # [None,None,3] X [H,W,3] -> [H,W,3] it is broaddcasted
            combined_flow = P_k_dot + inst_rotation_action # [H,W,3] + [H,W,3] -> [H,W,3] 
            
            influence_of_flow_on_a_point = torch.cross(P_k,combined_flow) # [H,W,3] X [H,W,3] -> [H,W,3] 
            tranformed_influence_in_ego_frame= torch.matmul(R_k[None,None],influence_of_flow_on_a_point[...,None]).squeeze(-1) # None,None,3,3  @ H,W,3 -> H,W,3
            # [H,W,3] @ [3,3].T = H,W,3 
            # Compute m_i for each pixel
            mki=tranformed_influence_in_ego_frame[masks[k]]
            temp_M=torch.einsum("ij,ik->jk", mki, mki) # output 3,3 matrix 
            M+=temp_M
            c += -1 * torch.matmul(temp_M, h_k.T)
            self.mi[k][masks[k]]=mki
            self.hk[k]=h_k
        
        self.M=M
        self.c=c
        
        self.t=self.compute_optimal_t(M,c)
        
        return self.mi , self.c , self.M ,self.hk ,self.t
    
    def compute_optimal_t(self,M,c):
        return torch.matmul(torch.linalg.pinv(M),c)
    

    def compute_optimal_eigen_val_t_degenerate(self,matrix):
    
        eigenvalues, eigenvectors = torch.linalg.eig(matrix)
        eigenvalues_real = eigenvalues.real
        positive_eigenvalues = [(val, idx) for idx, val in enumerate(eigenvalues_real) if val > 0]

        #TODO( PARTH): write code to catch this outside 
        if not positive_eigenvalues:
            raise ValueError("No positive eigenvalues found.")
        
        min_val, min_idx = min(positive_eigenvalues, key=lambda x: x[0].item())
        corresponding_vector = eigenvectors[:, min_idx].real
        
        return min_val, corresponding_vector

    
    def is_degenarate(self,M,epsilon=1e-6):
        return torch.abs(torch.linalg.det(M)) < epsilon

In [ ]:
#TESTING functions
obj=MultiCamEgoMotion()
obj.compute_data(torch.zeros((6,400,600,2),device=device),torch.zeros((6,400,600,2),device=device),torch.ones((6,4,4),device=device),torch.zeros((1,3),device=device),torch.ones((6,400,600),dtype=torch.bool,device=device))
print(obj.is_degenarate(obj.M))
print(obj.compute_optimal_t(obj.M,obj.c))

In [ ]:



matrix = torch.tensor([[4.0, 2.0,1.0], [1.0,2.0, 3.0],[1.0,2.0, 3.0]], dtype=torch.float32)
min_eigenvalue, eigenvector = obj.compute_optimal_eigen_val_t_degenerate(matrix)

print("Minimum Positive Eigenvalue:", min_eigenvalue.item())
print("Corresponding Eigenvector:", eigenvector)


In [34]:
from torch.autograd.functional import jacobian
torch.manual_seed(1)
size = 4096

In [34]:
#TODO(PARTH) Fix Theseus Solver Giving Issues 
"""

# Define a custom cost function for J1(w)
class J1CostFunction(th.CostFunction,MultiCamEgoMotion):
    def __init__(self, name,normalized_coords,flows,Extriniscs,cost_weight,w):
        """
        Args:
            flow_images (torch.Tensor): Optical flow images of shape [K, H, W, 2].
            R_k_list (list[torch.Tensor]): List of rotation matrices for each camera.
            b_k_list (list[torch.Tensor]): List of translation vectors for each camera.
            t (torch.Tensor): Translational velocity vector.
        """
        self.cost_weight = th.ScaleCostWeight(1.0)
        th.CostFunction.__init__(self, name=name, cost_weight=self.cost_weight)
        MultiCamEgoMotion.__init__(self)
        
        self.w =w
        self.register_optim_vars(["w"])
        
        self.normalized_coords=normalized_coords
        self.flow=flows
        self.Extriniscs=Extriniscs
        self.w_tensor=w.tensor
        
    
    def dim(self):
        """
        Return the dimensionality of the cost function.
        """
        # Example: return dimensionality of the input variable w
        return 3

    def jacobians(self):
        """
        Compute the Jacobian matrix of the cost function using PyTorch autograd.
        """
        #w = self.w
        #w = w.tensor.requires_grad_(True)  # Enable gradient computation for w

        # Compute the error
        
        with torch.enable_grad():
            J1 = self.error()
            
            #print(J1)

        
            grad = torch.autograd.grad(J1, self.w_tensor, retain_graph=False, allow_unused=False)
            print(grad)
        
        
        return grad[0], J1  # Directly return the gradient wrt w


    def _copy_impl(self, new_name):
        """
        Creates a copy of this cost function object.
        """
        return J1CostFunction(
            new_name,
            self.normalized_coords.clone(),
            self.flow.clone(),
            self.Extriniscs.clone(),
            self.cost_weight.copy(),self.w.copy()
        )

    def error(self):
        """
        Compute the residual error for J1(w).
        """
        # Extract optimization variable w
        w = self.w
        w.tensor.g
        self.w_tensor = w.tensor.requires_grad_(True)
        
        mi ,c , M, hk , t = self.compute_data(self.normalized_coords, self.flow, self.Extriniscs,self.w_tensor)
        
        if self.is_degenarate(M):
            self.degenrate_case=True 
        
        print(self.degenrate_case)
        #print("M is ", M)
        #print(c)
        M_reg = M + 1e-6 * torch.eye(M.size(-1), device=M.device)
        
        
        J1 = -1 * torch.dot(c.squeeze(-1), torch.matmul(torch.linalg.pinv(M_reg), c).squeeze(-1))
        J1 = torch.sum((mi * hk[:, None, None, :]).sum(dim=-1) ** 2)# mi- K,H,W,3  , hk :K,3
                
        return J1
"""

In [254]:
"""

# Solver Function
def solve_ego_motion_single_time_step(images_t0,images_t1,Extrinsics,Intrinsics, w_init):
    """
    Solve the ego-motion estimation problem using Theseus with image format [K, H, W, 2].
    """
    # Convert inputs to PyTorch tensors
    #np_img_t0=np.array([cv2.imread(img_name,0) for img_name in images_t0])
    #np_img_t1=np.array([[cv2.imread(img_name,0) for img_name in images_t1]])
    flow_images,_=compute_optical_flow(images_t0,images_t1,dense=True,vis=True,data_path="/home/parth/Desktop/NortheasternCourses/MobileRobotics/FinalProject/Multicam_EgoMotion/nuscenes_mini")
    normalized_coords,normalized_flow,clean_flow_mask = clean_optical_flow_to_normalized_plane_multiple_cameras(flow_images, np.array(Intrinsics))
    #flow_images = torch.tensor(flow_images, dtype=torch.float32) # [K, H, W, 2].
    normalized_flow=normalized_flow /0.05
    
    Extrinsics = torch.tensor(Extrinsics, dtype=torch.float32) #[K,4,4]
    Intrinsics = torch.tensor(Intrinsics, dtype=torch.float32)
    w_init = torch.tensor(w_init, dtype=torch.float32) # [3]
    normalized_coords =torch.tensor(normalized_coords, dtype=torch.float32)
    normalized_flow = torch.tensor(normalized_flow,dtype=torch.float32)

    # Define optimization variable
    print(w_init.shape)
    w = th.Vector(tensor=w_init, name="w")
    #w.tensor = w_init
    
    

    # Create the J1 cost function
    j1_cost = J1CostFunction(
        name="J1",
        normalized_coords=normalized_coords,
        flows=normalized_flow,
        Extriniscs=Extrinsics,
        cost_weight = 1,
        w=w,
        mask=clean_flow_mask
    )

    # Create the objective
    objective = th.Objective()
    objective.add(j1_cost) 
    
    theseus_inputs = {"w":w_init}
    objective.update(theseus_inputs)
    
    
    print("Batch size:", objective.batch_size)
    print("Cost weight:", j1_cost.cost_weight)
    print("Optim vars:", j1_cost.optim_vars)
    optimizer = th.LevenbergMarquardt(
    objective,
    linear_solver_cls=th.LUDenseSolver,
    linearization_cls=th.DenseLinearization,
    max_iterations=50,
    step_size=0.5,
    damping=0.1
    )



    # Solve the optimization problem
    optimizer.optimize(
       max_iterations=5,  # Maximum iterations
       verbose=True,  # Print optimization progress
    )

    print(j1_cost.M)
    print(j1_cost.c)
    t_optimal=j1_cost.compute_optimal_t(j1_cost.M,j1_cost.c).detach().numpy()
    w_optimal= w.tensor.detach()
        
        
    
    # # Return the optimized w
    return w_optimal , t_optimal

"""


In [ ]:
"""
w_init=np.array([[0.004477957729250193, 0.02866952493786812, 0.017157725989818573]])
w_optimal, t_optimal=solve_ego_motion_single_time_step(data[scene1][0]["image_list"][:4],data[scene1][1]["image_list"][:3],data[scene1][0]["extrinsics"][:3],data[scene1][0]["intrinsics"][:3],w_init)
print("w_optimal",w_optimal,"t_optimal",t_optimal)
"""

In [ ]:
print(data[scene1][0]["vel"],data[scene1][0]["rotation_rate"])

In [ ]:
print(data[scene1][0]["extrinsics"])

In [ ]:
#!pip install adabelief-pytorch

In [297]:
#PYTORCH SOLVER PART ( USE THIS INSTEAD OF THESEUS )
import torch
import torch.optim as optim


def solve_ego_motion_pytorch(images_t0, images_t1, Extrinsics, Intrinsics, w_init, max_iterations=10, learning_rate=0.01):
    """
    Solve the ego-motion estimation problem using PyTorch to optimize w.
    """
    # Compute optical flow and preprocess input
    flow_images, _ = compute_optical_flow(
        images_t0, images_t1, dense=False, vis=True,
        data_path=data_path
    )
    normalized_coords, normalized_flow ,clean_flow_mask= clean_optical_flow_to_normalized_plane_multiple_cameras(
        flow_images, np.array(Intrinsics)
    )
    
    obj=MultiCamEgoMotion()

    
    Extrinsics = torch.tensor(Extrinsics, dtype=torch.float32).to(device=device)  # [K, 4, 4]
    Intrinsics = torch.tensor(Intrinsics, dtype=torch.float32).to(device=device)
    normalized_coords = torch.tensor(normalized_coords, dtype=torch.float32).to(device=device)
    normalized_flow = torch.tensor(normalized_flow, dtype=torch.float32).to(device=device)
    w = torch.tensor(w_init, dtype=torch.float32, requires_grad=True,device= device)
    clean_flow_mask =torch.tensor(clean_flow_mask,dtype=torch.bool).to(device=device)

    
    #optimizer = optim.Adam([w], lr=learning_rate,weight_decay=0.1)
    #optimizer = SGD(  eps=1e-16, betas=(0.9, 0.999), weight_decay=1e-8)
    #optimizer=optimizer = optim.SGD([w], lr=0.01, momentum=0.9)
    #optimizer=torch.optim.RMSprop([w], lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0.01, momentum=0.9)
    optimizer=torch.optim.Adagrad([w], lr=0.01, weight_decay=0.01)


    
    def compute_loss(w):
        """
        Custom loss function that computes the cost based on J1.
        """
        mi ,c , M, hk , t= obj.compute_data(normalized_coords, normalized_flow, Extrinsics, w,clean_flow_mask)
        M_reg = M + 1e-6 * torch.eye(M.size(-1), device=M.device) # For numerical stability 
        
        #print("Is degenerate:", obj.is_degenarate(M_reg))
        if obj.degenrate_case:
            J1 = -1 * torch.dot(c.squeeze(-1), torch.matmul(torch.linalg.pinv(M), c).squeeze(-1))
            J1 += torch.sum((mi[clean_flow_mask] * hk[:, None, None, :]).sum(dim=-1) ** 2) 
        else:
            min_eigenvalue,translation_vec=obj.compute_optimal_eigen_val_t_degenerate(M)  # mi- K,H,W,3  , hk :K,3
            J1=min_eigenvalue
        return J1

    
    for iteration in range(max_iterations):
        optimizer.zero_grad()  # Reset gradients
        loss = compute_loss(w)  # Compute the loss
        loss.backward(retain_graph=True)  # Compute gradients
        optimizer.step()  # Update w

       
        if iteration % 5 == 0 or iteration == max_iterations - 1:
            print(f"Iteration {iteration}: Loss = {loss.item()}, w = {w}")

    t_optimal = obj.compute_optimal_t(obj.M,obj.c)
        
    final_t=t_optimal.detach().cpu().numpy()
        
    if obj.is_degenarate(obj.M).detach().cpu().numpy():    
        t_optimal = obj.compute_optimal_t(obj.M,obj.c)
        
        final_t=t_optimal.detach().cpu().numpy()
        #print(obj.M,obj.c)
    else:
        _,t_optimal=obj.compute_optimal_eigen_val_t_degenerate(obj.M)
        final_t=t_optimal.detach().cpu().numpy()
        
    # Return optimized w and t
    return obj.is_degenarate(obj.M).detach().cpu().numpy(), w.detach().cpu().numpy(), final_t



In [ ]:
import gc
with torch.no_grad():
    torch.cuda.empty_cache()
gc.collect()

In [ ]:
w_init=np.array([[0.004477957729250193, 0.02866952493786812, 0.017157725989818573]])
degenerate_sol,w_optima_per_frame, t_optimal_per_frame=solve_ego_motion_pytorch(data[scene1][1]["image_list"][:],data[scene1][2]["image_list"][:],data[scene1][0]["extrinsics"][:],data[scene1][0]["intrinsics"][:],w_init,max_iterations=55)
print("degenerate_sol",degenerate_sol,"w_optimal",w_optima_per_frame,"t_optimal",t_optimal_per_frame)


In [ ]:
dt=0.1
t_optimal_dt = t_optimal_per_frame/dt 
w_optimal_dt = w_optima_per_frame/dt
print("vel_optimal",t_optimal_dt.T, "w_optimal",w_optimal_dt)

In [ ]:
samples=4
for i in range(samples):
    print("velocity", data[scene1][i]["vel"]," roation rate",data[scene1][i]["rotation_rate"],(data[scene1][i+1]["timestamp"]-data[scene1][i]["timestamp"])/1e6)

In [ ]:
print("Delta Translation",t_optimal_dt[0]-data[scene1][0]["vel"],"Delta Rotation",w_optimal_dt[0]-data[scene1][0]["rotation_rate"])

In [ ]:
rotation_rate_pred=[]
translation_rate_pred=[]
rotation_rate_gt=[]
translation_rate_gt=[]

rotation_rate_error=[]
translation_rate_error=[]
degenerate_case=[]

w_init=np.array([[0.00, 0.0, 0.0]])

for scene in data.keys():
    
    w_init=np.array([[0.004477957729250193, 0.02866952493786812, 0.1]])
    print("Scene",scene)
    for traj_id in range(1,len(data[scene])):
        print("Trajectory id :",traj_id)
        degenrate_sol,w_optima_per_frame, t_optimal_per_frame=solve_ego_motion_pytorch(data[scene][traj_id-1]["image_list"][:],data[scene][traj_id]["image_list"][:],data[scene][traj_id]["extrinsics"][:],data[scene1][traj_id]["intrinsics"][:],w_init,max_iterations=15)
        
        dt=0.1
        t_optimal_dt = t_optimal_per_frame/dt
        w_optimal_dt = w_optima_per_frame[0]/dt
        
        
        print("w_optimal_frame",w_optima_per_frame,"t_optimal_frame",t_optimal_per_frame)
        print("w_optimal_dt",w_optimal_dt,"t_optimal_dt",t_optimal_dt)
        print("Delta Translation",t_optimal_dt-data[scene1][traj_id]["vel"],"Delta Rotation",w_optimal_dt-data[scene1][traj_id]["rotation_rate"])
        
        rotation_rate_pred.append(w_optimal_dt)
        translation_rate_pred.append(t_optimal_dt)
        rotation_rate_gt.append(data[scene1][traj_id]["rotation_rate"])
        translation_rate_gt.append(data[scene1][traj_id]["vel"])
        degenerate_case.append(degenrate_sol)
        if not degenerate_sol:
            w_init=w_optima_per_frame

        
        with torch.no_grad():
            torch.cuda.empty_cache()
        gc.collect()
    
    break



In [ ]:
print(degenerate_case)
print(np.sum(degenerate_case),len(degenerate_case))

In [ ]:
for i,rotation in enumerate(rotation_rate_gt):
    print(rotation)

In [ ]:
print(rotation_rate_pred[:])
plt.plot( np.array(rotation_rate_pred)[1:,2], marker='o',label="pred")
plt.plot( np.array(rotation_rate_gt)[1:,2], marker='o',label="gt")


plt.title("Rotation Rate Over Indices")
plt.xlabel("Index")
plt.ylabel("Rotation Rate")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
print(rotation_rate_pred[:])
plt.plot( np.array(rotation_rate_gt)[:,2], marker='o')
plt.plot( np.array(rotation_rate_gt)[:,1], marker='o')

plt.plot( np.array(rotation_rate_gt)[:,0], marker='o')

plt.title("Rotation Rate Over Indices")
plt.xlabel("Index")
plt.ylabel("Rotation Rate")
plt.grid(True)
plt.show()

In [ ]:
print(rotation_rate_pred[:])
plt.plot( np.array(translation_rate_pred)[:,0], marker='o')

plt.plot( np.array(translation_rate_gt)[:,0], marker='o')

plt.title("Translation Rate Over Indices")
plt.xlabel("Index")
plt.ylabel("Rotation Rate")
plt.grid(True)
plt.show()

In [ ]:
plt.plot( np.array(translation_rate_gt)[:,2], marker='o')
plt.plot( np.array(translation_rate_gt)[:,1], marker='o')

plt.plot( np.array(translation_rate_gt)[:,0], marker='o')

plt.title("Translation Rate Over Indices")
plt.xlabel("Index")
plt.ylabel("Rotation Rate")
plt.grid(True)
plt.show()

In [257]:
import numpy as np
import matplotlib.pyplot as plt

def rmse_error(x_pred, x_gt):
    
    if x_pred.shape != x_gt.shape:
        raise ValueError("Shapes of x_pred and x_gt must match")

    rmse = np.sqrt(np.mean((x_pred - x_gt) ** 2))
    return rmse

def distribution_error(x_pred, x_gt, num_bins=10):
    
    if x_pred.shape != x_gt.shape:
        raise ValueError("Shapes of x_pred and x_gt must match")

    abs_diff = np.linalg.norm(x_pred - x_gt, axis=1)
    hist, bin_edges = np.histogram(abs_diff, bins=num_bins, density=True)
    return bin_edges, hist

def plot_distributions(bin_edges, hist_values, title="Distribution Error", xlabel="Error", ylabel="Frequency"):
    
    plt.figure(figsize=(8, 5))
    plt.bar(bin_edges[:-1], hist_values, width=np.diff(bin_edges), align="edge", alpha=0.7, edgecolor="black")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.show()



In [ ]:
print(rmse_error( np.nan_to_num(np.array(rotation_rate_gt)), np.nan_to_num(np.array(rotation_rate_pred))[:,:]))
print(rmse_error( np.nan_to_num(np.array(rotation_rate_gt)[:,0]), np.nan_to_num(np.array(rotation_rate_pred))[:,0]))
print(rmse_error( np.nan_to_num(np.array(rotation_rate_gt)[:,1]), np.nan_to_num(np.array(rotation_rate_pred))[:,1]))
print(rmse_error( np.nan_to_num(np.array(rotation_rate_gt)[:,2]), np.nan_to_num(np.array(rotation_rate_pred))[:,2]))


print(rmse_error( np.nan_to_num(np.array(translation_rate_gt)[:,:]), np.nan_to_num(np.array(translation_rate_pred))[:,]))
print(rmse_error( np.nan_to_num(np.array(translation_rate_gt)[:,0]), np.nan_to_num(np.array(translation_rate_pred))[:,0]))
print(rmse_error( np.nan_to_num(np.array(translation_rate_gt)[:,1]), np.nan_to_num(np.array(translation_rate_pred))[:,1]))
print(rmse_error( np.nan_to_num(np.array(translation_rate_gt)[:,2]), np.nan_to_num(np.array(translation_rate_pred))[:,2]))


In [ ]:

x_pred = np.array(rotation_rate_gt)
x_gt = np.array(rotation_rate_pred)
rmse = rmse_error(x_pred, x_gt)
print("RMSE:", rmse)
bin_edges, hist_values = distribution_error(x_pred, x_gt, num_bins=20)
plot_distributions(bin_edges, hist_values)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def compute_cross_product(P_k, P_k_dot, omega_k):
    # Compute cross products
    cross_omega_Pk = np.cross(omega_k, P_k)
    inner_term = P_k_dot + cross_omega_Pk
    cross_Pk_inner = np.cross(P_k, inner_term)
    return cross_Pk_inner

def compute_Pk_dot(P_k, omega_k, t_k):
    # Compute P_k_dot using equation (3): P_k_dot = -omega_k x P_k - t_k
    cross_omega_Pk = np.cross(omega_k, P_k)
    P_k_dot = -cross_omega_Pk - t_k
    return P_k_dot

# Define example vectors
P_k = np.array([1.2, -3.1, 1.0])      # Example 3D point
omega_k = np.array([0.1, 0.2, 0.3])  # Angular velocity
t_k = np.array([0.5, -0.5, 0.5])     # Translational velocity

# Compute P_k_dot using the equation
P_k_dot = compute_Pk_dot(P_k, omega_k, t_k)

# Define a range of scalar multipliers for visualization
t_values = np.linspace(-1, 1, 100)

# Compute results for the cross product in each case
results = np.array([compute_cross_product(P_k, P_k_dot, omega_k) * t for t in t_values])
#print(results)
#print(t_values)

# Extract x, y, z components for 3D plotting
x_values = results[:, 0]
y_values = results[:, 1]
z_values = results[:, 2]

# Plot the 3D results
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the cross product
ax.plot(x_values, y_values, z_values, label='Pk x [ Pk_dot + omega_k x Pk ]', color='blue')

# Add vectors P_k, omega_k, and t_k to the plot
origin = np.array([0, 0, 0])
ax.quiver(*origin, *P_k, color='green', label='P_k', arrow_length_ratio=0.1)
ax.quiver(*origin, *omega_k, color='red', label='omega_k', arrow_length_ratio=0.1)
ax.quiver(*origin, *t_k, color='purple', label='t_k', arrow_length_ratio=0.1)

# Set labels and title
ax.set_title('3D Visualization of Cross Product with Vectors')
ax.set_xlabel('X Component')
ax.set_ylabel('Y Component')
ax.set_zlabel('Z Component')
ax.legend()
plt.show()
